In [25]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Markdown

In [10]:
from crewai import Agent, Task, Crew
from utils.j_utils import load_env_vars
from utils.ui_utils import load_json
from langchain_openai import ChatOpenAI
import langchain_groq
from langchain_groq import ChatGroq
secrets_path = '.streamlit/secrets.toml'
research_data_path = 'data/research_example.json'

In [4]:

groq_key = load_env_vars(secrets_path, "GROQ_API_KEY")
research_data = load_json(research_data_path)

In [13]:
#OPEN AI DATA
model = "gpt-3.5-turbo"
api_key = load_env_vars(secrets_path, "OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=api_key, model=model)

In [12]:
#GROQ DATA
model = "llama3-70b-8192"
api_key = load_env_vars(secrets_path, "GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model_name=model)

### Agents

In [21]:
planner = Agent(
    role=research_data['agents'][0]['role'],
    goal=research_data['agents'][0]['goal'],
    backstory=research_data['agents'][0]['backstory'],
    allow_delegation=False,
    verbose=True, 
    llm=llm
)
writer = Agent(
    role=research_data['agents'][1]['role'],
    goal=research_data['agents'][1]['goal'],
    backstory=research_data['agents'][1]['backstory'],
    allow_delegation=False,
    verbose=True, 
    llm=llm
)
editor = Agent(
    role=research_data['agents'][2]['role'],
    goal=research_data['agents'][2]['goal'],
    backstory=research_data['agents'][2]['backstory'],
    allow_delegation=False,
    verbose=True, 
    llm=llm
)

### Tasks

In [23]:
plan = Task(
    description=research_data['tasks'][0]['description'],
    expected_output=research_data['tasks'][0]['expected_output'],
    agent=planner
)
write = Task(
    description=research_data['tasks'][1]['description'],
    expected_output=research_data['tasks'][1]['expected_output'],
    agent=writer
)
edit= Task(
    description=research_data['tasks'][2]['description'],
    expected_output=research_data['tasks'][2]['expected_output'],
    agent=editor
)

### CREW

In [29]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=0
)

2024-06-10 14:31:56,637 - 8268270592 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [30]:
result = crew.kickoff(
    inputs={"topic": "Artificial Intelligents"}
)



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan Document

Title: The Rise of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Introduction of the latest trends and key players in the AI field

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning: Discuss the advancements in machine learning algorithms and their applications
- Natural Language Processing: Explore the latest developments in NLP and its impact on chatbots and virtual assistants
- Computer Vision: Highlight the progress in computer vision technology and its applications in image recognition and autonomous vehicles

2. Key Players in the AI Industry:
- Google: Discuss Google's AI initiatives such as Google Assistant and DeepMind
- Amazon: Explore Amazon's AI technologies like Alexa and recommendation systems
- Tesla: Highlight Tesla's use of

In [31]:
print(f"Model: {model}")
Markdown(result)

Model: gpt-3.5-turbo


# The Rise of Artificial Intelligence: Latest Trends and Key Players

## Introduction
Artificial Intelligence (AI) has revolutionized various industries, making processes more efficient and enabling innovative solutions. The latest trends in AI are shaping the future of technology, with key players driving advancements in machine learning, natural language processing, and computer vision.

## Latest Trends in Artificial Intelligence
### Machine Learning
One of the most significant trends in AI is the rapid advancement of machine learning algorithms. Companies are leveraging machine learning to improve decision-making processes, optimize operations, and personalize customer experiences. From recommendation systems to predictive analytics, machine learning is at the forefront of AI innovation.

### Natural Language Processing
Another key trend in AI is the development of natural language processing (NLP) technologies. NLP enables machines to understand and interpret human language, leading to the rise of chatbots and virtual assistants. The latest developments in NLP have enhanced conversational AI capabilities, making interactions more seamless and intuitive.

### Computer Vision
Advancements in computer vision technology have transformed the way machines perceive and analyze visual information. From image recognition to autonomous vehicles, computer vision is revolutionizing industries such as healthcare, automotive, and retail. The progress in computer vision algorithms is paving the way for enhanced safety, efficiency, and decision-making processes.

## Key Players in the AI Industry
### Google
Google is a major player in the AI industry, with initiatives such as Google Assistant and DeepMind pushing the boundaries of AI capabilities. Google's AI technologies are integrated into various products and services, enhancing user experiences and driving innovation across different sectors.

### Amazon
Amazon is at the forefront of AI innovation, with technologies like Alexa and recommendation systems reshaping the e-commerce landscape. By leveraging AI, Amazon provides personalized recommendations, streamlined shopping experiences, and efficient logistics operations. The company's AI initiatives continue to set new standards for customer engagement and satisfaction.

### Tesla
Tesla's use of AI in self-driving cars and energy optimization is setting new benchmarks for autonomous vehicles and sustainable energy solutions. By incorporating AI technologies into its products, Tesla is revolutionizing transportation and energy sectors, paving the way for a future powered by intelligent systems.

## Noteworthy News in Artificial Intelligence
### Recent Breakthroughs in AI Research
Researchers are constantly pushing the boundaries of AI through groundbreaking discoveries and innovations. From advancements in deep learning to ethical considerations in AI development, the latest news in AI research is shaping the future of technology and society.

### Impact of AI on Job Automation
As AI technologies continue to evolve, the impact on job automation and workforce changes is a significant consideration. Companies are adopting AI to streamline operations, leading to shifts in job roles and skill requirements. Understanding the implications of AI on the workforce is crucial for navigating the future of work.

### Ethical Considerations in AI Development
Ethical considerations in AI development and deployment are essential for ensuring responsible and sustainable AI solutions. From bias in algorithms to data privacy concerns, addressing ethical challenges in AI is crucial for building trust and transparency in AI systems.

## Conclusion
The rise of Artificial Intelligence is reshaping industries and driving innovation across various sectors. By exploring the latest trends and key players in the AI industry, we gain insights into the future of technology and the transformative power of intelligent systems. Stay informed about AI trends and engage in discussions to be part of the evolving landscape of Artificial Intelligence.